# Modeling — Avançado

Este notebook tem como objetivo **melhorar o desempenho do modelo baseline**
por meio de:
- engenharia leve de features
- modelos não lineares mais expressivos
- otimização sistemática de hiperparâmetros

O foco é **performance preditiva**, mantendo:
- split temporal
- ausência de data leakage
- reprodutibilidade

A análise detalhada dos resultados será feita no Notebook 05.


In [ ]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src import config
from src.data.load import load_raw_data
from src.data.clean import drop_invalid_rows, drop_leakage_columns, drop_missing
from src.features.target import build_delay_probability, binarize_target
from src.features.engineering import add_month_cyclical_features
from src.features.selection import get_advanced_features
from src.models.preprocessing import make_preprocessor
from src.models.train import train_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, classification_report


In [2]:
df = load_raw_data()
df.shape

(318017, 21)

In [8]:
df = df.copy()

df = df[df["arr_flights"] > 0]

df["delay_probability"] = df["arr_del15"] / df["arr_flights"]

threshold = df["delay_probability"].quantile(0.75)

df["target"] = (df["delay_probability"] >= threshold).astype(int)

df["target"].value_counts(normalize=True)

target
0    0.748479
1    0.251521
Name: proportion, dtype: float64

In [9]:
df = df.drop(columns=config.LEAKAGE_COLS, errors="ignore")

## Engenharia de features (leve)

Nesta etapa, são criadas apenas features:
- derivadas de tempo
- de baixo risco de leakage
- interpretáveis


In [10]:
# sazonalidade cíclica
df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)

In [11]:
FEATURE_COLS = [
    "year",
    "month_sin",
    "month_cos",
    "airport",
    "carrier",
    "arr_flights",
]

TARGET_COL = "target"

df = df[FEATURE_COLS + [TARGET_COL]].dropna()
df.shape

(317517, 7)

In [12]:
train_df = df[df["year"] < 2019]
test_df = df[df["year"] >= 2019]

X_train = train_df[FEATURE_COLS]
y_train = train_df[TARGET_COL]

X_test = test_df[FEATURE_COLS]
y_test = test_df[TARGET_COL]

In [14]:
categorical_cols = ["airport", "carrier"]
numeric_cols = ["year", "arr_flights", "month_sin", "month_cos"]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore",
         sparse_output=False), categorical_cols),
        ("num", "passthrough", numeric_cols),
    ]
)

## Random Forest

Modelo robusto, não linear, resistente a outliers e adequado
para capturar interações complexas.


In [15]:
rf = RandomForestClassifier(
    random_state=config.RANDOM_STATE,
    n_jobs=-1,
    class_weight="balanced"
)

In [23]:
param_grid = {
    "model__n_estimators": [300],
    "model__max_depth": [None, 20],
    "model__min_samples_leaf": [50],
    "model__max_features": ["sqrt"],
}


rf_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("model", rf),
    ]
)

In [24]:
grid = GridSearchCV(
    rf_pipeline,
    param_grid=param_grid,
    scoring="roc_auc",
    cv=3,                 # CV interno no treino
    n_jobs=1,
    verbose=2,
)

In [25]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] END model__max_depth=None, model__max_features=sqrt, model__min_samples_leaf=50, model__n_estimators=300; total time=  33.5s
[CV] END model__max_depth=None, model__max_features=sqrt, model__min_samples_leaf=50, model__n_estimators=300; total time=  32.7s
[CV] END model__max_depth=None, model__max_features=sqrt, model__min_samples_leaf=50, model__n_estimators=300; total time=  33.2s
[CV] END model__max_depth=20, model__max_features=sqrt, model__min_samples_leaf=50, model__n_estimators=300; total time=  24.2s
[CV] END model__max_depth=20, model__max_features=sqrt, model__min_samples_leaf=50, model__n_estimators=300; total time=  25.8s
[CV] END model__max_depth=20, model__max_features=sqrt, model__min_samples_leaf=50, model__n_estimators=300; total time=  27.4s


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...m_state=42))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'model__max_depth': [None, 20], 'model__max_features': ['sqrt'], 'model__min_samples_leaf': [50], 'model__n_estimators': [300]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'roc_auc'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",3
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and 

In [26]:
grid.best_params_

{'model__max_depth': 20,
 'model__max_features': 'sqrt',
 'model__min_samples_leaf': 50,
 'model__n_estimators': 300}

In [27]:
best_model = grid.best_estimator_

In [28]:
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
y_pred = best_model.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred_proba)
roc_auc

0.6652805691811392

In [29]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.78      0.82     57000
           1       0.27      0.43      0.33     11116

    accuracy                           0.72     68116
   macro avg       0.57      0.60      0.58     68116
weighted avg       0.78      0.72      0.74     68116



## Comparação com o baseline

O modelo Random Forest apresenta:
- maior capacidade de capturar interações
- melhor desempenho preditivo (ROC-AUC)
- maior custo computacional

A comparação detalhada com os modelos baseline
(Regressão Logística e Árvore rasa)
será realizada no Notebook 05.


## Conclusão do Notebook 04

Este notebook demonstrou que:
- engenharia leve de features melhora desempenho
- modelos não lineares capturam sinal adicional
- o problema se beneficia de maior expressividade do modelo

Próximo passo:
- análise detalhada de métricas
- interpretação do modelo
- avaliação de limitações e riscos
